In [30]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.impute import SimpleImputer
import seaborn as sns
from sklearn.metrics import  precision_score, recall_score, f1_score,balanced_accuracy_score
import seaborn as sns
from tqdm import tqdm
from utils.Preprocessing_utils import output_selection_prepro
from utils.heatmap_plots import plot_heat_map, plot_metrics
from predict_model import test_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from models.FeedForwardNet.model import Net


In [31]:
df = pd.read_csv('data/raw/dukecathr.csv')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [32]:

targets = list()
train_losses=[]
train_accu=[]
balanced_accuracy_list = []
precision_list = []
recall_list = []

In [33]:
def train_epoch(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for data in tqdm(dataloader):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        predicted = torch.round(outputs)  # Round the output probabilities to get binary predictions
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss=running_loss/len(dataloader)
    accu=100.*correct/total
    return train_loss, accu

In [34]:
def train_model(model, train_dataloader, criterion, optimizer, num_epochs=25):
    model = model.to(device)  # Move the model to the GPU

    for epoch in range(num_epochs):
        print(f'Epoch : {epoch+1}')
        train_loss, accu = train_epoch(model, train_dataloader, criterion, optimizer)
        train_accu.append(accu)
        train_losses.append(train_loss)
        print(f'Train Loss: {train_loss:.3f} | Train Accuracy: {accu:.3f}')


In [35]:
def evaluate_model(model, dataloader, criterion, target):
    model = model.to(device)  # Move the model to the GPU
    model.eval()  # Set model to evaluate mode
    running_loss=0
    correct=0
    total=0
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for data in tqdm(dataloader):
            inputs,labels=data[0].to(device),data[1].to(device)
            outputs=model(inputs)
            loss= criterion(outputs,labels)
            running_loss+=loss.item()
            predicted = torch.round(outputs)  # Round the output probabilities to get binary predictions
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # Store all labels and predictions for metric calculation
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
            
    test_loss=running_loss/len(dataloader)
    accu=100.*correct/total
    
    # Calculate metrics
    
    balance_acc = balanced_accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)

    # Append the metrics to the lists
    balanced_accuracy_list.append(balance_acc)
    precision_list.append(precision)
    recall_list.append(recall)
    targets.append(target)

    print(f'Test Loss: {test_loss:.3f} | Accuracy: {accu:.3f}')

In [36]:
def process_target(df, target):
   # Step 1: Preprocess the data
   X, y, num_cols, cat_cols = output_selection_prepro(df, target)

   # Split the data into train and test sets
   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
   
   Scaler = StandardScaler()
   X_train = Scaler.fit_transform(X_train)
   X_test = Scaler.transform(X_test)
   
   
   # Convert features and labels to tensors
   X_train_tensor = torch.tensor(X_train.astype(np.float32))
   X_test_tensor = torch.tensor(X_test.astype(np.float32))
   y_train_tensor = torch.tensor(y_train.values).float().unsqueeze(1)
   y_test_tensor = torch.tensor(y_test.values).float().unsqueeze(1)

   # Convert the data into PyTorch tensors and load them into a DataLoader
   train_data = TensorDataset(X_train_tensor, y_train_tensor)
   test_data = TensorDataset(X_test_tensor, y_test_tensor)

   batch_size = 2048
   train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
   test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

   # Define the model, loss function, and optimizer
   model = Net(X.shape[1])
   criterion = nn.BCELoss()
   optimizer = torch.optim.Adam(model.parameters(), lr=0.000125)

   # Train the model
   train_model(model, train_loader, criterion, optimizer, num_epochs=200)
   
   #evaluate the model
   evaluate_model(model, test_loader, criterion, target)
   

In [37]:
process_target(df, 'RCAST' )


Loading data from data/processed/RCAST/imputed_data_RCAST.csv...
No missing values found.
Final X columns: ['RDAYSFROMINDEX', 'RSEQCATHNUM', 'RSUBJID', 'YRCATH_G', 'AGE_G', 'GENDER', 'RACE_G', 'DPCABG', 'DPMI', 'DPPCI', 'HXANGINA', 'HXCEREB', 'HXCHF', 'HXCOPD', 'HXDIAB', 'HXHTN', 'HXHYL', 'HXMI', 'HXSMOKE', 'NUMPRMI', 'DIASBP_R', 'PULSE_R', 'SYSBP_R', 'CBRUITS', 'HEIGHT_R', 'S3', 'WEIGHT_R', 'CREATININE_R', 'HDL_R', 'LDL_R', 'TOTCHOL_R', 'CATHAPPR', 'DIAGCATH', 'INTVCATH', 'CORDOM', 'GRAFTST', 'LVEF_R', 'DAYS2LKA', 'DSCABG', 'DSMI', 'DSPCI', 'DSSTROKE', 'FUPROTCL']
Epoch : 1


100%|██████████| 27/27 [00:01<00:00, 25.06it/s]


Train Loss: 0.664 | Train Accuracy: 59.929
Epoch : 2


100%|██████████| 27/27 [00:00<00:00, 40.09it/s]


Train Loss: 0.619 | Train Accuracy: 66.522
Epoch : 3


...

Train Loss: 0.460 | Train Accuracy: 77.483
Epoch : 200


100%|██████████| 27/27 [00:00<00:00, 42.21it/s]


Train Loss: 0.459 | Train Accuracy: 77.399


100%|██████████| 7/7 [00:00<00:00, 27.54it/s]


Test Loss: 0.499 | Accuracy: 75.276


In [40]:
results = pd.DataFrame({'target' : targets , 'Balanced Accuracy': balanced_accuracy_list, 'Precision': precision_list, 'Recall': recall_list})
results

,target,Balanced Accuracy,Precision,Recall
0,RCAST,0.713307,0.762547,0.874985


In [39]:
# from sklearn.model_selection import train_test_split
# from torch.utils.data import TensorDataset, DataLoader
# from sklearn.preprocessing import LabelEncoder

# X, y, num_cols, cat_cols = output_selection_prepro(df, 'LADST')


# # Split data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# #scale

# # Standardize the features
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)
 

# # Convert data to tensors
# train_ds = TensorDataset(torch.tensor(X_train.astype(np.float32)), torch.tensor(y_train.values.astype(np.float32)))
# test_ds = TensorDataset(torch.tensor(X_test.astype(np.float32)), torch.tensor(y_test.values.astype(np.float32)))

# # Create data loaders
# batch_size = 1024
# train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
# test_dl = DataLoader(test_ds, batch_size=batch_size)

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(f"Using {device} for training.")

# # Instantiate the model
# model = Net(X.shape[1]).to(device)

# # Define loss function and optimizer
# criterion = nn.BCELoss().to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# epochs = 100

# # Training loop
# for epoch in range(epochs):
#     for i, (x_batch, y_batch) in enumerate(train_dl):
#         x_batch = x_batch.to(device)
#         y_batch = y_batch.to(device)  # Move y_batch to the GPU
#         # Forward pass
#         outputs = model(x_batch)
#         loss = criterion(outputs, y_batch.unsqueeze(1))
        
#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

# # Evaluation
# model.eval()
# with torch.no_grad():
#     all_outputs = []
#     all_labels = []
#     total_loss = 0
#     for images, labels in test_dl:
#         images = images.to(device)
#         labels = labels.to(device)  # Move labels to the GPU
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         all_outputs.extend(predicted.cpu().numpy())
#         all_labels.extend(labels.cpu().numpy())
#         loss = criterion(outputs, labels.unsqueeze(1))
#         total_loss += loss.item()

#     print('Test Loss: {}'.format(total_loss / len(test_dl)))

#     # Calculate metrics
#     balanced_acc = balanced_accuracy_score(all_labels, all_outputs)
#     precision = precision_score(all_labels, all_outputs, average='weighted')    
#     recall = recall_score(all_labels, all_outputs, average='weighted')
#     accuracy = accuracy_score(all_labels, all_outputs)
#     print('Balanced Accuracy: {}'.format(balanced_acc))
#     print('Precision: {}'.format(precision))
#     print('Recall: {}'.format(recall))
#     print('Accuracy: {}'.format(accuracy))